### Теория

[Документация](https://pytorch.org/docs/stable/nn.html#normalization-layers)

[Видео с объяснениями от Сергея Дубинина](https://www.youtube.com/watch?v=CSRu8byqxNs&list=PLBP4Q3FNSLK2rtGPBsK-aMAetYj-8yg_1&index=17&ab_channel=%D0%A1%D0%B5%D1%80%D0%B3%D0%B5%D0%B9%D0%94%D1%83%D0%B1%D0%B8%D0%BD%D0%B8%D0%BD)

BatchNorm, или Batch Normalization, это метод нормализации, применяемый в нейронных сетях для улучшения скорости обучения и стабильности. Он работает путем нормализации активаций каждого слоя для каждого мини-батча, вычитая среднее значение и деля на стандартное отклонение.

**Как работает BatchNorm:**

1. **Вычисление статистики мини-батча:** Для каждого признака (канала) в мини-батче вычисляется среднее значение и стандартное отклонение.

2. **Нормализация:**  Каждый признак в мини-батче нормализуется с использованием вычисленных статистики: `z = (x - mean) / std`, где `x` - исходное значение признака, `mean` - среднее значение по мини-батчу, `std` - стандартное отклонение по мини-батчу.  Это приводит данные к распределению с нулевым средним и единичной дисперсией.

3. **Масштабирование и сдвиг:**  Нормализованные значения затем масштабируются и сдвигаются с использованием обучаемых параметров `gamma` (гамма) и `beta` (бета):  `y = gamma * z + beta`. Это позволяет сети восстановить представление, которое могло быть потеряно при нормализации.


**Преимущества использования BatchNorm:**

* **Ускорение обучения:** BatchNorm позволяет использовать более высокие скорости обучения, что сокращает время обучения.
* **Снижение чувствительности к инициализации весов:** Модель становится менее чувствительной к начальным значениям весов.
* **Регуляризация:** BatchNorm действует как регуляризатор, уменьшая переобучение, особенно в небольших сетях, и иногда позволяет уменьшить необходимость в Dropout.
* **Стабилизация обучения:** BatchNorm помогает с проблемой исчезающего и взрывающегося градиента.
* **Улучшение качества модели:**  BatchNorm может привести к повышению точности модели.

**Нюансы:**
- Батчнорм можно применять, как перед активацией. так и после, как лучше, никто не скажет, необходимо экспериментировать.
- У слоя, к которому применяется нормализация, необходимо отключить смещение. Ошибки не будет, но лучше его убрать.

**Минусы использования батчнорм**
- BatchNorm() не стоит использовать вместе с Dropout()
- BatchNorm() плохо работает при маленьком размере batch-a
- BatchNorm() по разному работает при обучении и валидации, аналогично Dropout()

**Где применяется BatchNorm:**

BatchNorm обычно применяется после линейных слоев или сверточных слоев и перед функцией активации.  Однако, точное размещение может варьироваться в зависимости от архитектуры сети.


**BatchNorm в PyTorch:**

В PyTorch BatchNorm реализован в модулях `torch.nn.BatchNorm1d`, `torch.nn.BatchNorm2d` и `torch.nn.BatchNorm3d` для одномерных, двумерных и трехмерных данных соответственно.  Например, для двумерных данных (изображений) используется `torch.nn.BatchNorm2d`.  Важно правильно установить параметр `num_features`, который соответствует количеству каналов во входных данных.

In [25]:
# Пример использования nn.Dropout()

import torch.nn as nn

model = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.BatchNorm1d(20),  # BatchNorm() принимает 20 на вход из 1ого лин слоя
    nn.Linear(20, 10),
    nn.Softmax(dim=1)
)

### Импорты

In [27]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision.transforms import v2

import os
import matplotlib.pyplot as plt
import numpy as np

import json
from tqdm import tqdm
from PIL import Image

import time  # для замера времени

plt.style.use('dark_background')

### Рассмотрим на примере задачи классификации

#### Подготовка данных для обучения

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
### Пробуем для 'cuda'
class MNISTDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

        self.len_dataset = 0 # длина датасет
        self.data_list = [] # список кортежей путей до файла и позиции в onehot векторе

        # итерируемся по папке с основными файлами
        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = sorted(dir_list)
                self.class_to_idx = {
                    cls_name: i for i, cls_name in enumerate(self.classes)
                    }
                continue

            cls = path_dir.split('/')[-1]

            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)
                self.data_list.append((file_path, self.class_to_idx[cls]))

            self.len_dataset += len(file_list)

    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = Image.open(file_path)

        if self.transform is not None:
            sample = self.transform(sample)
            target = self.transform(target)

        return sample, target

In [4]:
# Преобразование для изображений
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.5, ), std=(0.5, ))
    ]
)

In [5]:
# создание датасетов
train_data = MNISTDataset('mnist/training', transform=transform)
test_data = MNISTDataset('mnist/testing', transform=transform)

In [6]:
train_data, val_data = random_split(train_data, [0.7, 0.3])

In [7]:
# Создание загрузчиков
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

#### Создание модели с nn.BatchNorm1d()

In [61]:
# создаем наш класс с BatchNorm1d()
class MyModelBN(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.layer_1 = nn.Linear(input, 256)
        self.layer_2 = nn.Linear(256, output)
        self.batchnorm = nn.BatchNorm1d(256)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.act(x)
        x = self.batchnorm(x)
        out = self.layer_2(x)

        return out

In [42]:
# создаем наш класс с dropout() для сравнения времени
class MyModelDO(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.layer_1 = nn.Linear(input, 256)
        self.layer_2 = nn.Linear(256, output)
        self.dropout = nn.Dropout(0.25)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.act(x)
        x = self.dropout(x)
        out = self.layer_2(x)

        return out

In [62]:
# инициализируем модель с nn.BatchNorm1d()
model_with_BatchNorm = MyModelBN(784, 10).to(device)
# инициализируем модель с dropout() 
model_with_Dropout = MyModelDO(784, 10).to(device)

In [63]:
# Проверяем правильность построения модели
input = torch.rand([16, 784], dtype=torch.float32).to(device)

out = model_with_BatchNorm(input)
out.shape    # (16,10)

torch.Size([16, 10])

In [64]:
# Проверяем правильность построения модели
input = torch.rand([16, 784], dtype=torch.float32).to(device)

out = model_with_Dropout(input)
out.shape    # (16,10)

torch.Size([16, 10])

### Тренеровка модели с Dropout()

Добавил в цикл обучения расчет времени на каждую эпоху, чтобы оценить "ускорение" от применения нормализации

In [55]:
# выбираем функцию потерь и оптимизатор градиентного спуска
loss_model = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model_with_Dropout.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=5)

In [56]:
EPOCHS = 10
train_loss = []
train_acc = []
val_loss = []
val_acc = []
lr_list = []
best_loss = None
count = 0

In [57]:
# Цикл обучения
for epoch in range(EPOCHS):
    
    start_time = time.time()  # Засекаем время начала эпохи
    
    # Тренировка модели
    model_with_Dropout.train()
    running_train_loss = []
    true_answer = 0
    # добавим трейн луп, чтобы видеть прогресс обучения модели
    train_loop = tqdm(train_loader, leave=False)
    for x, targets in train_loop:
        # Данные
        # (batch.size, 1, 28, 28) --> (batch.size, 784)
        x = x.reshape(-1, 28*28).to(device)
        # (batch.size, int) --> (batch.size, 10), dtype=float32
        targets = targets.reshape(-1).to(torch.int32)
        targets = torch.eye(10)[targets].to(device)

        # Прямой проход + расчет ошибки модели
        pred = model_with_Dropout(x)
        loss = loss_model(pred, targets)

        # Обратный проход
        opt.zero_grad()
        loss.backward()
        
        # Шаг оптимизации
        opt.step()

        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss)/len(running_train_loss)

        true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()

        train_loop.set_description(f"Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}")


    # Расчет значения метрики
    running_train_acc = true_answer / len(train_data)

    # Сохранение значения функции потерь и метрики
    train_loss.append(mean_train_loss)
    train_acc.append(running_train_acc)

    # Проверка модели (Валидация)
    model_with_Dropout.eval()
    with torch.no_grad():
        running_val_loss = []
        true_answer = 0
        for x, targets in val_loader:
            # Данные
            # (batch.size, 1, 28, 28) --> (batch.size, 784)
            x = x.reshape(-1, 28*28).to(device)
            # (batch.size, int) --> (batch.size, 10), dtype=float32
            targets = targets.reshape(-1).to(torch.int32)
            targets = torch.eye(10)[targets].to(device)

            # Прямой проход + расчет ошибки модели
            pred = model_with_Dropout(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss)/len(running_val_loss)

            true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()

        # Расчет значения метрики
        running_val_acc = true_answer / len(val_data)

        # Сохранение значения функции потерь и метрики
        val_loss.append(mean_val_loss)
        val_acc.append(running_val_acc)

        lr_scheduler.step(mean_val_loss)
        lr = lr_scheduler._last_lr[0]
        lr_list.append(lr)

        print(f"Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}, train_acc={running_train_acc:.4f}, val_loss={mean_val_loss:.4f}, val_acc={running_val_acc:.4f}")

        # добавляем две проверки, для сохранения лучшей модели
        if best_loss is None:
            best_loss = mean_val_loss
      
        if mean_val_loss < best_loss:
            best_loss = mean_val_loss
            
            # если модель улучшила свои показатели, то отсчет эпох пойдет заново
            # обнуляем счетчик
            count = 0
            
            # так же сохраняем словарь в случае улучшения модели
            checkpoint = {
                'state_model': model_with_Dropout.state_dict(),
                'state_opt': opt.state_dict(),
                'state_lr_scheduler': lr_scheduler.state_dict(),
                'loss':{
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'best_loss': best_loss
                },
                'metric':{
                    'train_acc': train_acc,
                    'val_acc': val_acc
                },
                'lr': lr_list,
                'epoch':{
                    'EPOCHS': EPOCHS,
                    'save_epoch': epoch
                }
            }
    
            
    
            torch.save(checkpoint, f'model_state_dict_epoch_{epoch+1}.pt')
            print(f"На эпохе: {epoch+1}, сохранена модель со значением функции потерь на валидаци: {mean_val_loss:.4f}", end='\n\n')

        # условие, для остановки обучения по достижению счетчиком определенного значения!
        if count >= 10:
            print(f'\033[31mОбучение остановлено на {epoch + 1} эпохе.\033[0m')
            break
            
        # в конце каждой эпохи увеличиваем счетчик на 1
        count += 1

    # Засекаем время конца эпохи
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Время на эпоху {epoch + 1}: {epoch_time:.2f} секунд.")


Epoch [1/10], train_loss=0.4371, train_acc=0.8694, val_loss=0.2426, val_acc=0.9281
Время на эпоху 1: 19.59 секунд.


Epoch [2/10], train_loss=0.2334, train_acc=0.9297, val_loss=0.1632, val_acc=0.9510
На эпохе: 2, сохранена модель со значением функции потерь на валидаци: 0.1632

Время на эпоху 2: 19.62 секунд.


Epoch [3/10], train_loss=0.1805, train_acc=0.9463, val_loss=0.1430, val_acc=0.9578
На эпохе: 3, сохранена модель со значением функции потерь на валидаци: 0.1430

Время на эпоху 3: 19.47 секунд.


Epoch [4/10], train_loss=0.1581, train_acc=0.9522, val_loss=0.1267, val_acc=0.9627
На эпохе: 4, сохранена модель со значением функции потерь на валидаци: 0.1267

Время на эпоху 4: 20.19 секунд.


Epoch [5/10], train_loss=0.1376, train_acc=0.9577, val_loss=0.1277, val_acc=0.9616
Время на эпоху 5: 20.16 секунд.


Epoch [6/10], train_loss=0.1267, train_acc=0.9617, val_loss=0.1166, val_acc=0.9642
На эпохе: 6, сохранена модель со значением функции потерь на валидаци: 0.1166

Время на эпоху 6: 20.04 секунд.


Epoch [7/10], train_loss=0.1174, train_acc=0.9633, val_loss=0.1190, val_acc=0.9662
Время на эпоху 7: 19.86 секунд.


Epoch [8/10], train_loss=0.1077, train_acc=0.9662, val_loss=0.0980, val_acc=0.9714
На эпохе: 8, сохранена модель со значением функции потерь на валидаци: 0.0980

Время на эпоху 8: 19.98 секунд.


Epoch [9/10], train_loss=0.1033, train_acc=0.9663, val_loss=0.1091, val_acc=0.9683
Время на эпоху 9: 19.64 секунд.


Epoch [10/10], train_loss=0.0989, train_acc=0.9689, val_loss=0.0962, val_acc=0.9723
На эпохе: 10, сохранена модель со значением функции потерь на валидаци: 0.0962

Время на эпоху 10: 20.13 секунд.


### Тренеровка модели с BatchNorm()

In [65]:
# выбираем функцию потерь и оптимизатор градиентного спуска
loss_model = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model_with_BatchNorm.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=5)

In [66]:
EPOCHS = 10
train_loss = []
train_acc = []
val_loss = []
val_acc = []
lr_list = []
best_loss = None
count = 0

In [67]:
# Цикл обучения
for epoch in range(EPOCHS):
    
    start_time = time.time()  # Засекаем время начала эпохи
    
    # Тренировка модели
    model_with_BatchNorm.train()
    running_train_loss = []
    true_answer = 0
    # добавим трейн луп, чтобы видеть прогресс обучения модели
    train_loop = tqdm(train_loader, leave=False)
    for x, targets in train_loop:
        # Данные
        # (batch.size, 1, 28, 28) --> (batch.size, 784)
        x = x.reshape(-1, 28*28).to(device)
        # (batch.size, int) --> (batch.size, 10), dtype=float32
        targets = targets.reshape(-1).to(torch.int32)
        targets = torch.eye(10)[targets].to(device)

        # Прямой проход + расчет ошибки модели
        pred = model_with_BatchNorm(x)
        loss = loss_model(pred, targets)

        # Обратный проход
        opt.zero_grad()
        loss.backward()
        
        # Шаг оптимизации
        opt.step()

        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss)/len(running_train_loss)

        true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()

        train_loop.set_description(f"Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}")


    # Расчет значения метрики
    running_train_acc = true_answer / len(train_data)

    # Сохранение значения функции потерь и метрики
    train_loss.append(mean_train_loss)
    train_acc.append(running_train_acc)

    # Проверка модели (Валидация)
    model_with_BatchNorm.eval()
    with torch.no_grad():
        running_val_loss = []
        true_answer = 0
        for x, targets in val_loader:
            # Данные
            # (batch.size, 1, 28, 28) --> (batch.size, 784)
            x = x.reshape(-1, 28*28).to(device)
            # (batch.size, int) --> (batch.size, 10), dtype=float32
            targets = targets.reshape(-1).to(torch.int32)
            targets = torch.eye(10)[targets].to(device)

            # Прямой проход + расчет ошибки модели
            pred = model_with_BatchNorm(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss)/len(running_val_loss)

            true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()

        # Расчет значения метрики
        running_val_acc = true_answer / len(val_data)

        # Сохранение значения функции потерь и метрики
        val_loss.append(mean_val_loss)
        val_acc.append(running_val_acc)

        lr_scheduler.step(mean_val_loss)
        lr = lr_scheduler._last_lr[0]
        lr_list.append(lr)

        print(f"Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}, train_acc={running_train_acc:.4f}, val_loss={mean_val_loss:.4f}, val_acc={running_val_acc:.4f}")

        # добавляем две проверки, для сохранения лучшей модели
        if best_loss is None:
            best_loss = mean_val_loss
      
        if mean_val_loss < best_loss:
            best_loss = mean_val_loss
            
            # если модель улучшила свои показатели, то отсчет эпох пойдет заново
            # обнуляем счетчик
            count = 0
            
            # так же сохраняем словарь в случае улучшения модели
            checkpoint = {
                'state_model': model_with_BatchNorm.state_dict(),
                'state_opt': opt.state_dict(),
                'state_lr_scheduler': lr_scheduler.state_dict(),
                'loss':{
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'best_loss': best_loss
                },
                'metric':{
                    'train_acc': train_acc,
                    'val_acc': val_acc
                },
                'lr': lr_list,
                'epoch':{
                    'EPOCHS': EPOCHS,
                    'save_epoch': epoch
                }
            }
    
            
    
            torch.save(checkpoint, f'model_state_dict_epoch_{epoch+1}.pt')
            print(f"На эпохе: {epoch+1}, сохранена модель со значением функции потерь на валидаци: {mean_val_loss:.4f}", end='\n\n')

        # условие, для остановки обучения по достижению счетчиком определенного значения!
        if count >= 10:
            print(f'\033[31mОбучение остановлено на {epoch + 1} эпохе.\033[0m')
            break
            
        # в конце каждой эпохи увеличиваем счетчик на 1
        count += 1

    # Засекаем время конца эпохи
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Время на эпоху {epoch + 1}: {epoch_time:.2f} секунд.")

Epoch [1/10], train_loss=0.3847, train_acc=0.8898, val_loss=0.3352, val_acc=0.9040
Время на эпоху 1: 19.95 секунд.


Epoch [2/10], train_loss=0.2990, train_acc=0.9135, val_loss=0.2694, val_acc=0.9211
На эпохе: 2, сохранена модель со значением функции потерь на валидаци: 0.2694

Время на эпоху 2: 19.58 секунд.


Epoch [3/10], train_loss=0.2697, train_acc=0.9231, val_loss=0.2518, val_acc=0.9272
На эпохе: 3, сохранена модель со значением функции потерь на валидаци: 0.2518

Время на эпоху 3: 19.81 секунд.


Epoch [4/10], train_loss=0.2549, train_acc=0.9253, val_loss=0.2466, val_acc=0.9287
На эпохе: 4, сохранена модель со значением функции потерь на валидаци: 0.2466

Время на эпоху 4: 20.31 секунд.


Epoch [5/10], train_loss=0.2383, train_acc=0.9309, val_loss=0.2475, val_acc=0.9291
Время на эпоху 5: 19.66 секунд.


Epoch [6/10], train_loss=0.2303, train_acc=0.9316, val_loss=0.2469, val_acc=0.9233
Время на эпоху 6: 19.53 секунд.


Epoch [7/10], train_loss=0.2192, train_acc=0.9349, val_loss=0.2357, val_acc=0.9302
На эпохе: 7, сохранена модель со значением функции потерь на валидаци: 0.2357

Время на эпоху 7: 20.09 секунд.


Epoch [8/10], train_loss=0.2130, train_acc=0.9369, val_loss=0.2321, val_acc=0.9315
На эпохе: 8, сохранена модель со значением функции потерь на валидаци: 0.2321

Время на эпоху 8: 20.14 секунд.


Epoch [9/10], train_loss=0.2067, train_acc=0.9387, val_loss=0.2240, val_acc=0.9344
На эпохе: 9, сохранена модель со значением функции потерь на валидаци: 0.2240

Время на эпоху 9: 20.09 секунд.


Epoch [10/10], train_loss=0.1996, train_acc=0.9400, val_loss=0.2274, val_acc=0.9366
Время на эпоху 10: 20.07 секунд.


### Выводы сравнения скорости работы с BatchNorm() и DropOut()

Ну в целом, не сказать, что модель с BatchNorm тренеруется быстрее модели с DropOut, вероятно, данное преимущество раскрывается на существенно более Больших моделях и других данных.

Так же, возможно, что некорректно замерять скорость именно эпохи, возможно стоит замерять скорость выполнения именно тренировочной части эпохи... Еще до валидации

Чтож, будем эксперементировать в будущем =)